![CC Project 2](https://pbs.twimg.com/media/FTXYRxhWQAEwMrg?format=jpg&name=medium)

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Part 1: Import Data and Analyze
<b> `Motorcycle Bike Dataset:` </b> The dataset is about motorcycles different features. We collect the data using a scrapper and the data information is in polish language. There are 40 numbers of features that contain information like motor model, year, engine type, color, brand, etc and 7481 number of observations. The Target attribute is Price so, based on attributes we predict the price of the Motor cycle.
In the first part we Analyze the data first we import three libraries. A library is a tool that you can use to make a specific job. First of all, we import the numpy library used for multidimensional array then import the pandas library used to import the dataset and in last we import matplotlib library used for plotting the graph. Then we import dataset using Pandas Dataframe after that we analyze the data like number of features, features type, data size etc.

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### 1. Import Libraries

In [51]:
import os
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)

### 1.1 Create a Bucket and Set the Region in S3

#### 1.1.1: Set the region of the instance

In [52]:
my_region = boto3.session.Session().region_name 
print(my_region)

us-east-1


#### 1.1.2: Create a S3 Bucket to store data for model

In [53]:
bucket1 = 'bikepriceapplication'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket1 )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


### 1.2 Upload the Data and store into S3 Data Bucket

#### 1.2.1: Set the path where the data will save

In [54]:
prefix1 = 'Dataset'
output_path1 ='s3://{}/{}/output'.format(bucket1, prefix1)
print(output_path1)

s3://bikepriceapplication/Dataset/output


#### 1.2.2: Upload the Data and Store in S3 Bucket

In [109]:
df = pd.read_csv('scrapped_23052022.csv')
df.head()

,Unnamed: 0,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Rok produkcji,Przebieg,Pojemność skokowa,Moc,Rodzaj napędu,Typ Silnika,Rodzaj paliwa,Skrzynia biegów,Typ nadwozia,Kolor,Możliwość finansowania,Faktura VAT,Leasing,Kraj pochodzenia,Bezwypadkowy,Stan,price,Numer rejestracyjny pojazdu,Zarejestrowany w Polsce,Rodzaj koloru,Pierwsza rejestracja,Serwisowany w ASO,Pierwszy właściciel,Gwarancja dealerska (w cenie),Tuning,VAT marża,Okres gwarancji producenta,Liczba pozostałych rat,Miesięczna rata,Opłata początkowa,Wartość wykupu,lub do (przebieg km),Uszkodzony,VIN,Zarejestrowany jako zabytek
0,0,Firmy,Motocykle,Kymco,MXU,2021,1 km,695 cm3,40 KM,Wał kardana,Czterosuwowy,Benzyna,Automatyczna,Quad,Czarny,Tak,Tak,Tak,Inny,Tak,Nowe,45900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Firmy,Motocykle,Segway,Fugleman UT10,2021,1 km,1 000 cm3,105 KM,Wał kardana,Czterosuwowy,Benzyna,Bezstopniowa,Quad,Inny kolor,Tak,Tak,Tak,NaN,Tak,Nowe,76990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Osoby prywatnej,Motocykle,Yamaha,FZ,2010,33 668 km,998 cm3,150 KM,Łańcuch,Czterosuwowy,Benzyna,Manualna,Naked,Biały,NaN,NaN,NaN,Francja,NaN,Używane,20500,PK 6785,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Osoby prywatnej,Motocykle,Ducati,Multistrada,2010,56 000 km,NaN,NaN,NaN,NaN,Benzyna,NaN,Turystyczny,Biały,NaN,NaN,NaN,NaN,NaN,Używane,32000,NaN,NaN,Metalik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Osoby prywatnej,Motocykle,Kawasaki,ZX,2007,16 000 km,250 cm3,NaN,NaN,NaN,Benzyna,NaN,Sportowy,Niebieski,NaN,NaN,NaN,NaN,Tak,Używane,4999,NaN,Tak,Metalik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.2.3:Store the data in data S3 Bucket for training the model

In [6]:
boto3.Session().resource('s3').Bucket(bucket1).Object(os.path.join(prefix1, 'scrapped_23052022.csv')).upload_file('scrapped_23052022.csv')
s3_data = sagemaker.TrainingInput(s3_data='s3://{}/{}/'.format(bucket1, prefix1), content_type='csv')

In [7]:
s3_data

### 3. Analyze the Data

#### 3.1 How many number of features and observation

In [8]:
print(f'Number of Observations: {df.shape[0]}')
print(f'Number of Features: {df.shape[1]}')

Number of Observations: 7481
Number of Features: 40


#### 3.2 Features that we have in this data

In [9]:
df.columns

Index(['Unnamed: 0', 'Oferta od', 'Kategoria', 'Marka pojazdu',
       'Model pojazdu', 'Rok produkcji', 'Przebieg', 'Pojemność skokowa',
       'Moc', 'Rodzaj napędu', 'Typ Silnika', 'Rodzaj paliwa',
       'Skrzynia biegów', 'Typ nadwozia', 'Kolor', 'Możliwość finansowania',
       'Faktura VAT', 'Leasing', 'Kraj pochodzenia', 'Bezwypadkowy', 'Stan',
       'price', 'Numer rejestracyjny pojazdu', 'Zarejestrowany w Polsce',
       'Rodzaj koloru', 'Pierwsza rejestracja', 'Serwisowany w ASO',
       'Pierwszy właściciel', 'Gwarancja dealerska (w cenie)', 'Tuning',
       'VAT marża', 'Okres gwarancji producenta', 'Liczba pozostałych rat',
       'Miesięczna rata', 'Opłata początkowa', 'Wartość wykupu',
       'lub do (przebieg km)', 'Uszkodzony', 'VIN',
       'Zarejestrowany jako zabytek'],
      dtype='object')

#### 3.3 Type of Features

In [10]:
df.dtypes

Unnamed: 0                         int64
Oferta od                         object
Kategoria                         object
Marka pojazdu                     object
Model pojazdu                     object
Rok produkcji                      int64
Przebieg                          object
Pojemność skokowa                 object
Moc                               object
Rodzaj napędu                     object
Typ Silnika                       object
Rodzaj paliwa                     object
Skrzynia biegów                   object
Typ nadwozia                      object
Kolor                             object
Możliwość finansowania            object
Faktura VAT                       object
Leasing                           object
Kraj pochodzenia                  object
Bezwypadkowy                      object
Stan                              object
price                              int64
Numer rejestracyjny pojazdu       object
Zarejestrowany w Polsce           object
Rodzaj koloru   

#### 3.4 Data Info

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7481 entries, 0 to 7480
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     7481 non-null   int64  
 1   Oferta od                      7481 non-null   object 
 2   Kategoria                      7481 non-null   object 
 3   Marka pojazdu                  7481 non-null   object 
 4   Model pojazdu                  7481 non-null   object 
 5   Rok produkcji                  7481 non-null   int64  
 6   Przebieg                       7420 non-null   object 
 7   Pojemność skokowa              6915 non-null   object 
 8   Moc                            5542 non-null   object 
 9   Rodzaj napędu                  6182 non-null   object 
 10  Typ Silnika                    6276 non-null   object 
 11  Rodzaj paliwa                  7481 non-null   object 
 12  Skrzynia biegów                6371 non-null   o

#### 3.4 Data Size

In [12]:
df.size

299240

#### 3.5 Predictor and Target Attribute

In [110]:
# Predictor Attributes
col = 'price'
X = df.loc[:, df.columns != col]
X.head()

,Unnamed: 0,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Rok produkcji,Przebieg,Pojemność skokowa,Moc,Rodzaj napędu,Typ Silnika,Rodzaj paliwa,Skrzynia biegów,Typ nadwozia,Kolor,Możliwość finansowania,Faktura VAT,Leasing,Kraj pochodzenia,Bezwypadkowy,Stan,Numer rejestracyjny pojazdu,Zarejestrowany w Polsce,Rodzaj koloru,Pierwsza rejestracja,Serwisowany w ASO,Pierwszy właściciel,Gwarancja dealerska (w cenie),Tuning,VAT marża,Okres gwarancji producenta,Liczba pozostałych rat,Miesięczna rata,Opłata początkowa,Wartość wykupu,lub do (przebieg km),Uszkodzony,VIN,Zarejestrowany jako zabytek
0,0,Firmy,Motocykle,Kymco,MXU,2021,1 km,695 cm3,40 KM,Wał kardana,Czterosuwowy,Benzyna,Automatyczna,Quad,Czarny,Tak,Tak,Tak,Inny,Tak,Nowe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Firmy,Motocykle,Segway,Fugleman UT10,2021,1 km,1 000 cm3,105 KM,Wał kardana,Czterosuwowy,Benzyna,Bezstopniowa,Quad,Inny kolor,Tak,Tak,Tak,NaN,Tak,Nowe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Osoby prywatnej,Motocykle,Yamaha,FZ,2010,33 668 km,998 cm3,150 KM,Łańcuch,Czterosuwowy,Benzyna,Manualna,Naked,Biały,NaN,NaN,NaN,Francja,NaN,Używane,PK 6785,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Osoby prywatnej,Motocykle,Ducati,Multistrada,2010,56 000 km,NaN,NaN,NaN,NaN,Benzyna,NaN,Turystyczny,Biały,NaN,NaN,NaN,NaN,NaN,Używane,NaN,NaN,Metalik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Osoby prywatnej,Motocykle,Kawasaki,ZX,2007,16 000 km,250 cm3,NaN,NaN,NaN,Benzyna,NaN,Sportowy,Niebieski,NaN,NaN,NaN,NaN,Tak,Używane,NaN,Tak,Metalik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
# Target Attributes
Y = df.iloc[:, 21]
Y

0       45900
1       76990
2       20500
3       32000
4        4999
        ...  
7476    27900
7477    57900
7478    11990
7479    15900
7480    25500
Name: price, Length: 7481, dtype: int64

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Part 2: Data Preprocessing
Data Preprocessing is a very important part of Building the Model. In this part, first, we select the features that are important for the application. In our case, we select the top 15 features from 40 and ignore the rest of the features. After that, we handled some misclassification of the object type. Then, we check the missing value in that 15 features and we find some missing values to handle we use the mode method for object value, and for the numerical value, we use the mean strategy. After that, we implement onehoteconder method to convert our obj value into numerical value because the machine learning model doesn't accept string value directly therefore we converted. And in the last part of Data Preprocessing, we split our data into training and test. And the ratio of train and test is 80% data for training and 20% for testing.

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### 2.1 Select Top Features

In [113]:
X = df.iloc[:, 5:9]
X.head()

,Rok produkcji,Przebieg,Pojemność skokowa,Moc
0,2021,1 km,695 cm3,40 KM
1,2021,1 km,1 000 cm3,105 KM
2,2010,33 668 km,998 cm3,150 KM
3,2010,56 000 km,NaN,NaN
4,2007,16 000 km,250 cm3,NaN


### 2.2 Handled the Missing Values

#### 2.2.1 Select categorical variables(obj) 

In [114]:
cat_vars = X.select_dtypes(include='object')
cat_vars.head()

,Przebieg,Pojemność skokowa,Moc
0,1 km,695 cm3,40 KM
1,1 km,1 000 cm3,105 KM
2,33 668 km,998 cm3,150 KM
3,56 000 km,NaN,NaN
4,16 000 km,250 cm3,NaN


#### 2.2.2 Select those who has missing variables(obj) 

In [60]:
isnull_per=cat_vars.isnull().mean()*100
miss_vars = isnull_per[isnull_per >0].keys()
miss_vars

Index(['Przebieg', 'Pojemność skokowa', 'Moc', 'Rodzaj napędu', 'Typ Silnika',
       'Skrzynia biegów'],
      dtype='object')

#### 2.2.3 Apply mode method to fill missing values(obj) 

In [115]:
X['Przebieg'] = X['Przebieg'].fillna(X['Przebieg'].mode()[0])

In [116]:
X['Pojemność skokowa'] = X['Pojemność skokowa'].fillna(df['Pojemność skokowa'].mode()[0])

In [117]:
X['Moc'] = X['Moc'].fillna(X['Moc'].mode()[0])

In [64]:
X['Rodzaj napędu'] = X['Rodzaj napędu'].fillna(X['Rodzaj napędu'].mode()[0])

In [65]:
X['Typ Silnika'] = X['Typ Silnika'].fillna(X['Typ Silnika'].mode()[0])

In [66]:
X['Typ Silnika'] = X['Typ Silnika'].fillna(X['Typ Silnika'].mode()[0])

In [67]:
X['Skrzynia biegów'] = X['Skrzynia biegów'].fillna(X['Skrzynia biegów'].mode()[0])

#### 2.2.4 Check the missing value after mode method 

In [118]:
X.isnull().sum()

Rok produkcji        0
Przebieg             0
Pojemność skokowa    0
Moc                  0
dtype: int64

### 2.3 Handled Missclassfication Object Type 

#### 2.3.1 Convert Str to Int which was misclassify

In [119]:
X['Przebieg'] = X['Przebieg'].str.extract(r'(\d+)').astype(int)

In [120]:
X['Pojemność skokowa'] = X['Pojemność skokowa'].str.extract(r'(\d+)').astype(int)

In [121]:
X['Moc'] = X['Moc'].str.extract(r'(\d+)').astype(int)

In [122]:
X

,Rok produkcji,Przebieg,Pojemność skokowa,Moc
0,2021,1,695,40
1,2021,1,1,105
2,2010,33,998,150
3,2010,56,125,15
4,2007,16,250,15
...,...,...,...,...
7476,2021,1,500,47
7477,2015,1,1,131
7478,2022,1,125,15
7479,2020,19,650,80


#### 2.3.2 Check type of feature after handled 

In [123]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7481 entries, 0 to 7480
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Rok produkcji      7481 non-null   int64
 1   Przebieg           7481 non-null   int64
 2   Pojemność skokowa  7481 non-null   int64
 3   Moc                7481 non-null   int64
dtypes: int64(4)
memory usage: 233.9 KB


### 2.4 OneHotEncoder 

#### 2.4.1 Import library from sklearn

In [71]:
from sklearn.preprocessing import OneHotEncoder

#### 2.4.2 Initialize the OneHotEncoder

In [75]:
one_hot_encoder = OneHotEncoder(sparse=False,)


#### 2.4.3 Fit the OHE

In [76]:
one_hot_encoder_var = one_hot_encoder.fit_transform(X)

In [82]:
one_hot_encoder_ = one_hot_encoder.fit(X)

In [77]:
one_hot_encoder_var

array([[1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

### 2.5 Split the Dataset

In [124]:
from sklearn.model_selection import train_test_split

#### 2.5.1 Split the Dataset into Train and Test

In [36]:
Y = df.iloc[:, 21]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

#### 2.5.2 Training Data

In [37]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)

X_train (5984, 1556)
y_train (5984,)


#### 2.5.3 Testing Data

In [38]:
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_test (1497, 1556)
y_test (1497,)


![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Part 3: Build the SVM Model
In this part we build the SVM regression model which is one of the most important part of our application. And we build Suppport Vector Machine from scratch and deploy end to end to model.

Support Vector Machines are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. In Support Vector Regression, the straight line that is required to fit the data is referred to as hyperplane.

The objective of a support vector machine algorithm is to find a hyperplane in an n-dimensional space that distinctly classifies the data points. The data points on either side of the hyperplane that are closest to the hyperplane are called Support Vectors. These influence the position and orientation of the hyperplane and thus help build the SVM.

https://medium.com/machine-learning-researcher/support-vector-machine-a57e575b05bb

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### 3.1 Build the Support Vector Machine Model

In [42]:
import tensorflow

In [39]:
from sklearn.svm import SVR
class _SVR(object):
    def __init__(self, epsilon=0.5):
        self.epsilon = epsilon
        
    def fit(self, X, y, epochs=100, learning_rate=0.1):
        self.sess = tf.Session()
        
        feature_len = X.shape[-1] if len(X.shape) > 1 else 1
        
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        if len(y.shape) == 1:
            y = y.reshape(-1, 1)
        
        self.X = tf.placeholder(dtype=tf.float32, shape=(None, feature_len))
        self.y = tf.placeholder(dtype=tf.float32, shape=(None, 1))
        
        self.W = tf.Variable(tf.random_normal(shape=(feature_len, 1)))
        self.b = tf.Variable(tf.random_normal(shape=(1,)))
        
        self.y_pred = tf.matmul(self.X, self.W) + self.b
        
        #self.loss = tf.reduce_mean(tf.square(self.y - self.y_pred))
        #self.loss = tf.reduce_mean(tf.cond(self.y_pred - self.y < self.epsilon, lambda: 0, lambda: 1))
        
        # Second part of following equation, loss is a function of how much the error exceeds a defined value, epsilon
        # Error lower than epsilon = no penalty.
        self.loss = tf.norm(self.W)/2 + tf.reduce_mean(tf.maximum(0., tf.abs(self.y_pred - self.y) - self.epsilon))
       # self.loss = tf.reduce_mean(tf.maximum(0., tf.abs(self.y_pred - self.y) - self.epsilon))
        
        opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        opt_op = opt.minimize(self.loss)

        self.sess.run(tf.global_variables_initializer())
        
        for i in range(epochs):
            loss = self.sess.run(
                self.loss, 
                {
                    self.X: X,
                    self.y: y
                }
            )
            print("{}/{}: loss: {}".format(i + 1, epochs, loss))
            
            self.sess.run(
                opt_op, 
                {
                    self.X: X,
                    self.y: y
                }
            )
            
        return self
    
        def predict(self, X, y=None):
            if len(X.shape) == 1:
                X = X.reshape(-1, 1)
            
            y_pred = self.sess.run(
                self.y_pred, 
                {
                    self.X: X 
                }
            )
            return y_pred

#### 3.1.1 Init the Model

In [40]:
model = SVR(kernel = 'rbf')

#### 3.1.2 Fit the Data to the Model

In [41]:
model.fit(X_train, y_train)

SVR()

#### 3.1.3 Predict the Test Data

In [46]:
y_predict = model.predict(X_test)
y_predict

array([19029.99098247, 18933.9182797 , 18917.95147325, ...,
       18948.8094085 , 19035.79666296, 18895.75001974])

In [48]:
input1 = ['Firmy', 'Motocykle', 'Kymco', 'MXU', 2021,  1, 695, 40, 'Wał kardana', 'Czterosuwowy', 'Benzyna', 'Automatyczna', 'Quad', 'Czarny']

In [49]:
one_hot_encoder.transform([input1])

array([[1., 0., 1., ..., 0., 0., 0.]])

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Part 4: Evaluate the Result
In this part, we evaluate the Support Vector Machine Model. And for evaluation we will use three metrics. <br>
`R Square`: R Square measures how much variability in dependent variable can be explained by the model. <br>
`Mean Square Error`: MSE is calculated by the sum of square of prediction error which is real output minus predicted output and then divide by the number of data points. It gives you an absolute number on how much your predicted results deviate from the actual number. <br>
`Mean Absolute Error`: Mean Absolute Error(MAE) is similar to Mean Square Error(MSE). However, instead of the sum of square of error in MSE, MAE is taking the sum of the absolute value of error.

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### 4.1: Evaluate the Model Performance

In [47]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#### 4.1.1: Calculate the Results

In [48]:
# R Square
R_Square = r2_score(y_predict, y_test)
print('R Square:', R_Square) 

# Mean Square Error
MSE = mean_squared_error(y_predict, y_test)
print('Mean Squared Error:', MSE)

# Mean Square Error
MAE = mean_absolute_error(y_predict, y_test)
print('Mean Absolute Error:', MAE)

R Square: -392860.1998408903
Mean Squared Error: 981360801.804831
Mean Absolute Error: 19451.57029283147


![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Part 5: Save the Model
In this part, we save our model. To do so we use pickle library and later we use this model to build application.

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### 5.1 Save the SVM and One_Hot_Encoder Model

#### 5.1.1: Import Pickle Library to Save the File

In [49]:
import pickle

#### 5.1.2: Save the Model and Store into s3 bucket

In [51]:
# Save the Model
Support_Vector_Machine = open("model.pkl","wb")          
pickle.dump(model,Support_Vector_Machine)                                  
Support_Vector_Machine.close() 

In [59]:
# Save the OHE
one_hot_encoder_variable = open("OHE.pkl","wb")        
pickle.dump(one_hot_encoder_var,one_hot_encoder_variable)                                  
one_hot_encoder_variable.close()

In [53]:
my_region = boto3.session.Session().region_name 
print(my_region)

us-east-1


In [54]:
bucket2 = 'bikemodelsave'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket2)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [55]:
prefix2 = 'Save Model'
output_path2 ='s3://{}/{}/output'.format(bucket2, prefix2)
print(output_path2)

s3://bikemodelsave/Save Model/output


In [63]:
boto3.Session().resource('s3').Bucket(bucket2).Object(os.path.join(prefix2, 'OHE.pkl')).upload_file('OHE.pkl')
s3_model = sagemaker.TrainingInput(s3_data='s3://{}/{}/'.format(bucket2, prefix2), content_type='csv')
s3_model 

In [64]:
boto3.Session().resource('s3').Bucket(bucket2).Object(os.path.join(prefix2, 'model.pkl')).upload_file('model.pkl')
s3_model = sagemaker.TrainingInput(s3_data='s3://{}/{}/'.format(bucket2, prefix2), content_type='csv')
s3_model

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)